In [1]:
import numpy as np
import pandas as pd
import os
import random
import copy

# Calculation of Relative Strength Index (RSI)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

dirname = os.path.abspath('')
data_path = os.path.join(dirname, 'Data')

stocks_df = pd.read_csv(data_path + '\\' + 'stocks_with_next_changes.csv', encoding='utf-8')

stocks = stocks_df['persian_symbol'].unique().tolist()
print(len(stocks))


373


In [2]:
indicators_df = pd.read_csv(data_path + '\\' + 'selected_stocks_with_indicators.csv', encoding='utf-8')
print(indicators_df.columns)

Index(['persian_symbol', 'pd_trade_date', 'market_value', 'first_price',
       'final_price', 'last_trade_price', 'number_of_trades', 'volume',
       'value', 'min_price', 'max_price', 'yesterday_price', 'change',
       'next_change', 'Momentum_1D', 'RSI_5D', 'Volume_plain'],
      dtype='object')


In [3]:
selected_symbols_df = pd.read_csv(data_path + '\\' + 'regression_benchmark.csv', encoding='utf-8')
selected_symbols = selected_symbols_df['symbol'].tolist()
print(selected_symbols)

['شفن', 'بفجر', 'غالبر', 'وآذر', 'دپارس', 'ملت', 'فلامي', 'قصفها', 'پكوير', 'سكرد', 'داسوهح', 'دالبر', 'كاوهح', 'غبهنوش', 'پاسا', 'وملي', 'سخوز', 'وپاسار', 'وخارزم', 'تمحركه']


In [4]:
stocks_df['Momentum_1D'] = 0

def rsi(values):
    up = values[values > 0].mean()
    down = -1 * values[values < 0].mean()
    return 100 * up / (up + down)

for stock in selected_symbols:
    stocks_df.loc[stocks_df['persian_symbol'] == stock, 'Momentum_1D'] = (
        stocks_df.loc[stocks_df['persian_symbol'] == stock, 'final_price'] -
        stocks_df.loc[stocks_df['persian_symbol'] == stock, 'final_price'].shift(1)
    ).fillna(0)

    stocks_df.loc[stocks_df['persian_symbol'] == stock, 'RSI_5D'] = \
        stocks_df.loc[stocks_df['persian_symbol'] == stock, 'Momentum_1D'].rolling(center=False, window=5). \
        apply(rsi).fillna(0)

stocks_df.to_csv(
    data_path + '\\' + 'selected_stocks_with_indicators.csv', encoding='utf-8-sig', index=False)

print('done')



done


In [5]:
# Calculation of Volume (Plain)

indicators_df['Volume_plain'] = indicators_df['volume'].fillna(0)

indicators_df.to_csv(
    data_path + '\\' + 'selected_stocks_with_indicators.csv', encoding='utf-8-sig', index=False)

print('done!')

done!


In [6]:
indicators_df['BB_Middle_Band'] = 0
indicators_df['BB_Upper_Band'] = 0
indicators_df['BB_Lower_Band'] = 0


# Calculation of Bollinger Bands
def bbands(price, length=5, numsd=2):
    ''' returns average, upper band, and lower band'''
    #ave = pd.stats.moments.rolling_mean(price,length)
    ave = price.rolling(window = length, center = False).mean()
    #sd = pd.stats.moments.rolling_std(price,length)
    sd = price.rolling(window = length, center = False).std()
    upband = ave + (sd*numsd)
    dnband = ave - (sd*numsd)
    return np.round(ave,3), np.round(upband,3), np.round(dnband,3)

for stock in selected_symbols:
    indicators_df.loc[indicators_df['persian_symbol'] == stock, 'BB_Middle_Band'],\
    indicators_df.loc[indicators_df['persian_symbol'] == stock, 'BB_Upper_Band'], \
    indicators_df.loc[indicators_df['persian_symbol'] == stock, 'BB_Lower_Band'] = \
        bbands(indicators_df.loc[indicators_df['persian_symbol'] == stock, 'final_price'], length=5, numsd=1)

indicators_df.fillna(0, inplace=True)

indicators_df.to_csv(
    data_path + '\\' + 'selected_stocks_with_indicators.csv', encoding='utf-8-sig', index=False)

print('done')

done


In [7]:
indicators_df['Aroon_Oscillator'] = 0

# Calculation of Aroon Oscillator
def aroon(df, tf=5):
    aroonup = []
    aroondown = []
    x = tf
    while x< len(df['pd_trade_date']):
        aroon_up = ((df['max_price'][x-tf:x].tolist().index(max(df['max_price'][x-tf:x])))/float(tf))*100
        aroon_down = ((df['min_price'][x-tf:x].tolist().index(min(df['min_price'][x-tf:x])))/float(tf))*100
        aroonup.append(aroon_up)
        aroondown.append(aroon_down)
        x+=1
    return aroonup, aroondown

for stock in selected_symbols:
    list_of_zeros = [0] * 5
    up, down = aroon(indicators_df.loc[indicators_df['persian_symbol'] == stock])
    aroon_list = [x - y for x, y in zip(up,down)]
    if len(aroon_list)==0:
        aroon_list = [0] * indicators_df.loc[indicators_df['persian_symbol'] == stock].shape[0]
        indicators_df.loc[indicators_df['persian_symbol'] == stock, 'Aroon_Oscillator'] = aroon_list
    else:
        indicators_df.loc[indicators_df['persian_symbol'] == stock, 'Aroon_Oscillator'] = list_of_zeros+aroon_list

indicators_df.fillna(0, inplace=True)

indicators_df.to_csv(data_path + '\\' + 'selected_stocks_with_indicators.csv', encoding='utf-8-sig', index=False)

print('done')

done


In [8]:
indicators_df['PVT'] = 0
# Calculation of Price Volume Trend
# PVT = [((CurrentClose - PreviousClose) / PreviousClose) x Volume] + PreviousPVT

for stock in selected_symbols:
    indicators_df.loc[indicators_df['persian_symbol'] == stock, 'PVT'] = \
        (indicators_df.loc[indicators_df['persian_symbol'] == stock, 'Momentum_1D']/
         indicators_df.loc[indicators_df['persian_symbol'] == stock, 'final_price'].shift(1))\
        * indicators_df.loc[indicators_df['persian_symbol'] == stock, 'volume']

    indicators_df.loc[indicators_df['persian_symbol'] == stock, 'PVT'] = \
        indicators_df.loc[indicators_df['persian_symbol'] == stock, 'PVT']\
        - indicators_df.loc[indicators_df['persian_symbol'] == stock, 'PVT'].shift(1)

    indicators_df.loc[indicators_df['persian_symbol'] == stock, 'PVT'] = \
        indicators_df.loc[indicators_df['persian_symbol'] == stock, 'PVT'].fillna(0)

indicators_df.to_csv(
    data_path + '\\' + 'selected_stocks_with_indicators.csv', encoding='utf-8-sig', index=False)
print('done')



done


In [9]:
# Calculation of Acceleration Bands
indicators_df['AB_Middle_Band'] = 0
indicators_df['aupband'] = 0
indicators_df['AB_Upper_Band'] = 0
indicators_df['adownband'] = 0
indicators_df['AB_Lower_Band'] = 0

def abands(df):
    for stock in selected_symbols:
        #df['AB_Middle_Band'] = pd.rolling_mean(df['final_price'], 5)
        df.loc[df['persian_symbol'] == stock, 'AB_Middle_Band'] = \
            df.loc[df['persian_symbol'] == stock, 'final_price'].rolling(window = 5, center=False).mean()

        # High * ( 1 + 4 * (High - Low) / (High + Low))
        df.loc[df['persian_symbol'] == stock, 'aupband'] = \
            df.loc[df['persian_symbol'] == stock, 'max_price'] * (
                    1 + 4 * (df.loc[df['persian_symbol'] == stock, 'max_price'] -
                             df.loc[df['persian_symbol'] == stock, 'min_price']) /
                    (df.loc[df['persian_symbol'] == stock, 'max_price'] +
                     df.loc[df['persian_symbol'] == stock, 'min_price'])
            )
        df.loc[df['persian_symbol'] == stock, 'AB_Upper_Band'] = df.loc[df['persian_symbol'] == stock, 'aupband'].\
            rolling(window=5, center=False).mean()

        # Low *(1 - 4 * (High - Low)/ (High + Low))
        df.loc[df['persian_symbol'] == stock, 'adownband'] = \
            df.loc[df['persian_symbol'] == stock, 'min_price'] * (
                    1 - 4 * (df.loc[df['persian_symbol'] == stock, 'max_price'] -
                             df.loc[df['persian_symbol'] == stock, 'min_price']) /
                    (df.loc[df['persian_symbol'] == stock, 'max_price'] +
                     df.loc[df['persian_symbol'] == stock, 'min_price'])
            )

        df.loc[df['persian_symbol'] == stock, 'AB_Lower_Band'] = \
            indicators_df.loc[indicators_df['persian_symbol'] == stock, 'adownband'].\
                rolling(window=5, center=False).mean()

abands(indicators_df)

indicators_df.fillna(0, inplace=True)

indicators_df.to_csv(
    data_path + '\\' + 'selected_stocks_with_indicators.csv', encoding='utf-8-sig', index=False)

print('done!')



done!


In [10]:
indicators_df['STOK'] = 0
# Calculation of Stochastic Oscillator (%K and %D)

def STOK(df, n):
    for stock in selected_symbols:
        df.loc[df['persian_symbol'] == stock, 'STOK'] = ((df.loc[df['persian_symbol'] == stock, 'final_price'] -
                                                          df.loc[df['persian_symbol'] == stock, 'min_price'].
                                                          rolling(window=n, center=False).mean()) /
                                                         (df.loc[df['persian_symbol'] == stock, 'max_price'].
                                                          rolling(window=n, center=False).max() -
                                                          df.loc[df['persian_symbol'] == stock, 'min_price'].
                                                          rolling(window=n, center=False).min())) * 100

        df.loc[df['persian_symbol'] == stock, 'STOD'] = df.loc[df['persian_symbol'] == stock, 'STOK'].\
            rolling(window = 3, center=False).mean()

STOK(indicators_df, 4)
indicators_df.fillna(0, inplace=True)

indicators_df.to_csv(data_path + '\\' + 'selected_stocks_with_indicators.csv', encoding='utf-8-sig', index=False)
print('done')



done


In [11]:
indicators_df['Chaikin_MF'] = 0
# Calculation of Chaikin Money Flow
def CMFlow(df, tf, stock):
    CHMF = []
    MFMs = []
    MFVs = []
    x = tf

    while x < len(df.loc[df['persian_symbol'] == stock, 'pd_trade_date']):
        PeriodVolume = 0
        volRange = df.loc[(df['persian_symbol'] == stock) & (df.index.isin(df.index[x-tf:x])), 'volume']
        for eachVol in volRange:
            PeriodVolume += eachVol

        MFM = ((df.loc[(df['persian_symbol'] == stock) & (df.index == x), 'final_price'] -
                df.loc[(df['persian_symbol'] == stock) & (df.index == x), 'min_price']) -
               (df.loc[(df['persian_symbol'] == stock) & (df.index == x), 'max_price'] -
                df.loc[(df['persian_symbol'] == stock) & (df.index == x), 'final_price'])) / \
              (df.loc[(df['persian_symbol'] == stock) & (df.index == x), 'max_price'] -
               df.loc[(df['persian_symbol'] == stock) & (df.index == x), 'min_price'])

        MFV = MFM*PeriodVolume
        MFMs.append(MFM)
        MFVs.append(MFV)
        x+=1

    y = tf
    while y < len(MFVs):
        PeriodVolume = 0
        volRange = df.loc[(df['persian_symbol'] == stock) & (df.index.isin(df.index[x-tf:x])), 'volume']
        for eachVol in volRange:
            PeriodVolume += eachVol
        consider = MFVs[y-tf:y]
        tfsMFV = 0

        for eachMFV in consider:
            tfsMFV += eachMFV

        tfsCMF = tfsMFV/PeriodVolume
        CHMF.append(tfsCMF)
        y+=1
    return CHMF

i = 0
for stock in selected_symbols:
    print(i, stock)
    i += 1
    listofzeros = [0] * 40
    CHMF = CMFlow(indicators_df, 20, stock)
    if len(CHMF)==0:
        CHMF = [0] * indicators_df.shape[0]
        indicators_df.loc[indicators_df['persian_symbol'] == stock, 'Chaikin_MF'] = CHMF
    else:
        indicators_df.loc[indicators_df['persian_symbol'] == stock, 'Chaikin_MF'] = listofzeros+CHMF

indicators_df.fillna(0, inplace=True)
indicators_df.to_csv(
    data_path + '\\' + 'selected_stocks_with_indicators.csv', encoding='utf-8-sig', index=False)

print('done!')



0 شفن
1 بفجر
2 غالبر
3 وآذر
4 دپارس
5 ملت
6 فلامي
7 قصفها
8 پكوير
9 سكرد
10 داسوهح
11 دالبر
12 كاوهح
13 غبهنوش
14 پاسا
15 وملي
16 سخوز
17 وپاسار
18 وخارزم
19 تمحركه
done!


c:\users\kia\documents\mypycharm\bachelor_project\venv\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
c:\users\kia\documents\mypycharm\bachelor_project\venv\lib\site-packages\pandas\core\internals\blocks.py:831: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  dtype, _ = maybe_promote(np.array(value).dtype)
c:\users\kia\documents\mypycharm\bachelor_project\venv\lib\site-packages\pandas\core\internals\blocks.py:848: VisibleDeprecationWarning: Creating an ndarray fro